<a href="https://colab.research.google.com/github/SilvanCodes/ag_wiehe/blob/main/1001genomes_PAM_GPN_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.0 MB/s eta 0:00:00


In [ ]:
import time
import requests
from collections import defaultdict
from io import StringIO

from tqdm import tqdm
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq, MutableSeq

In [ ]:
HMA4_gene_id = "AT2G19110.1"
amino_acid_substitution = "missense_variant"

In [ ]:
class DefaultDict(defaultdict):
    def __missing__(self, key):
        return self.default_factory(key)

def get_accession_gene_fasta(gene):
  def get_gene(accession):
    fasta = requests.get(f"https://tools.1001genomes.org/api/v1/pseudogenomes/strains/{accession}/gids/{gene}")
    return fasta.text
  return get_gene


In [ ]:
HMA4_fasta_cache = DefaultDict(get_accession_gene_fasta(HMA4_gene_id))

In [ ]:
aa_sub_snps = requests.get(f"https://tools.1001genomes.org/api/v1.1/variants.json?type=snps;accs=all;gid={HMA4_gene_id};effect={amino_acid_substitution}")
df = pd.DataFrame(aa_sub_snps.json()['data'], columns=["chromosome", "position", "accession", "reference", "variant", "-", "impacts", "effects"])
df

,chromosome,position,accession,reference,variant,-,impacts,effects
0,2,8279523,9653,T,G,40,MODERATE,missense_variant
1,2,8279523,9655,T,G,40,MODERATE,missense_variant
2,2,8279523,9661,T,G,40,MODERATE,missense_variant
3,2,8279523,9968,T,G,40,MODERATE,missense_variant
4,2,8279539,9525,T,A,40,MODERATE,missense_variant
...,...,...,...,...,...,...,...,...
2328,2,8286155,9845,G,T,40,MODERATE,missense_variant
2329,2,8286155,9886,G,T,40,MODERATE,missense_variant
2330,2,8286155,9888,G,T,40,MODERATE,missense_variant
2331,2,8286155,9894,G,T,40,MODERATE,missense_variant


In [ ]:
def get_start_end_from_seq(seq):
  range = seq.id.split("|")[4]
  _chrom, range = range.split(":")
  start, end = range.split("..")
  return int(start), int(end)

In [ ]:
def get_aa_substitution_pair(variant, record, in_gene_position):
  in_condon_position = in_gene_position % 3

  codon_start = (in_gene_position // 3) * 3
  codon = MutableSeq(record.seq[codon_start:codon_start+3])

  variant_aa = codon.translate()

  codon[in_condon_position] = variant["reference"]
  reference_aa = codon.translate()

  return reference_aa, variant_aa

In [ ]:
def build_variant_substitution_data(variant, fasta_cache):
  fasta = fasta_cache[variant["accession"]]
  record = SeqIO.read(StringIO(fasta), "fasta")

  start, _ = get_start_end_from_seq(record)
  in_gene_position = variant["position"] - start

  if variant["variant"] == record.seq[in_gene_position]:

    snp_centered_dna_window = record.seq[in_gene_position-256:in_gene_position+257]

    reference_aa, variant_aa = get_aa_substitution_pair(variant, record, in_gene_position)

    return [str(reference_aa), str(variant_aa), str(snp_centered_dna_window)]
  else:
    return ["-", "-", "-"]

In [ ]:
df[:2]

,chromosome,position,accession,reference,variant,-,impacts,effects
0,2,8279523,9653,T,G,40,MODERATE,missense_variant
1,2,8279523,9655,T,G,40,MODERATE,missense_variant


In [ ]:
results = []
for variant in tqdm(df[:100].to_dict(orient="records")):
    results.append(build_variant_substitution_data(variant, HMA4_fasta_cache))
    time.sleep(0.51)

100%|██████████| 100/100 [01:34<00:00,  1.05it/s]


In [ ]:
build_variant_substitution_data(df.iloc[29], HMA4_fasta_cache)

AssertionError: 

In [ ]:
weird_variant = df.iloc[29]

fasta = HMA4_fasta_cache[weird_variant["accession"]]
record = SeqIO.read(StringIO(fasta), "fasta")

start, _ = get_start_end_from_seq(record)
in_gene_position = weird_variant["position"] - start

print(record.seq[in_gene_position])
print(weird_variant)


G
chromosome                   2
position               8280931
accession                 9511
reference                    G
variant                      T
-                           40
impacts               MODERATE
effects       missense_variant
Name: 29, dtype: object


In [ ]:
results = pd.DataFrame(results, columns=["reference_aa", "variant_aa", "snp_centered_dna_window"])
results

,reference_aa,variant_aa,snp_centered_dna_window
0,L,V,AAAAGTAAACATTTTCAATAAGAAAATACAAGACCCATACCGAAAG...
1,L,V,AAAAGTAAACATTTTCAATAAGAAANNNNNNNNNNNNNNNNNNNNG...
2,L,V,AAAAGTAAACATTTTCAATAAGAAAATACAAGACCCATACCGAAAG...
3,L,V,NNAAGTAAANNNNTNNAATAANAAAATACAAGACCCATNCCGAAAG...
4,F,Y,AATAAGAAAANNNNNNNNNNNNNNCGNAAGTTTNTTNNNNANAAAA...
...,...,...,...
95,K,M,TATAACAATTGTGAAATCTCTTGCTATTTTTATAAATGATTTTGAA...
96,Y,F,CAATTGTGAAATCTCTTGCTATTTTTATAAATGATTTTGAAGTTGA...
97,-,-,-
98,-,-,-


In [ ]:
(results["reference_aa"] == "-").sum()

np.int64(51)

In [ ]:
from google.colab import files
results.to_csv("results.csv", index=False)
files.download("results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
results["snp_centered_dna_window"].apply(lambda x: len(x))

,snp_centered_dna_window
0,513
1,513


In [ ]:
first_result = df.loc[4].to_dict()
first_result

{'chromosome': 2,
 'position': 8279539,
 'accession': 9525,
 'reference': 'T',
 'variant': 'A',
 '-': 40,
 'impacts': 'MODERATE',
 'effects': 'missense_variant'}

In [ ]:
get_accession_gene_fasta(HMA4_gene_id)(first_result["accession"])

'>MPI-GMI|Ath-1001-Genomes|pseudo-genome|9525|Chr2:8278881..8286445|Col-0_gi:AT2G19110.1|V0.2\nCTACGTTCCTAACACTTCTCTCAACCTTTATCTGATCGCACCAAACCAGTTTTTTCGCATCGGCTNCTTCCTTTTGCTACTAGCTCTCCTCTCTTCTCCGGTNTTTTTGTCTCNCTTCTTAATTCACACAGATTTCATGATAAGTGATGATCTATAACAAGACGCTAACTCTTCTCTTGCATTTCTCGTGTTTTCATTTTCTTGTTACGCCAAATTTATCCCTTCAAAATCNNTTTTTTATGNGTATAGAATCCAAATAATAAGTAAAAGCTGATTCGTCTTCTTCCACTTAACACAAGTAAGCAGTGAGAGGGTGAAGATTTTTCTTTAGGAAAACAAAGAGAGTGAAGATATTTTTTGGCTTGATCTCAACATTATTTTTTCNTAAAAGTAAACATTTTCAATAAGAAAANNNNNNNNNNNNNNCGNAAGTTTNTTNNNNANAAAAAAAAAAAGGNTTTGANCTGTTTCATGATAATGATAACNAAAAAAGTTTTTGCTTTCTTNTTTTTTTTCCTCCGCAAAACAGTCTNAAAGTATAACCAAAAAGCCTATAAATCAATATAATTTGTTGTTTTGATTTACGTTTTACNGAAAATGGCGTTACAAAACAAAGAAGAAGAGAAAAAGAAAGTGAAGAAGTTGCAAAAGAGTTACTACGATGTTCTCGGAATCTGTTGTACATCGGAAGTTCCTATAATCGAGAATATTCTCAAGTCACTTGACGGCGTTAAAGAATATTCCGTCATCGTTCCCTCGAGAACCGTGNTTGTTGTTCACGACAGNNTCCTCATCTCTCCCTTCCNAATTGGTAAATNTTTTTTTTCTTTNTGATAATAAAGNTTTTTTNNNNNAAAAAAATTGGTAAATCATTATAANTAAATAGTTATTTAANATTTCTCTAA

In [ ]:
HMA4_fasta_cache = DefaultDict(get_accession_gene_fasta(HMA4_gene_id))

In [ ]:
get_aa_substitution_pair(first_result)

(MutableSeq('Y'), MutableSeq('F'))

In [ ]:
fasta = get_accession_gene_fasta(first_result["accession"], HMA4_gene_id)

In [ ]:
record = SeqIO.read(StringIO(fasta), "fasta")
record

SeqRecord(seq=Seq('CTACGTTCCTAACANTTCTCTCAACCTTTATCTGATCGCACCAAACCAGTTTTT...CCT'), id='MPI-GMI|Ath-1001-Genomes|pseudo-genome|9653|Chr2:8278881..8286445|Col-0_gi:AT2G19110.1|V0.2', name='MPI-GMI|Ath-1001-Genomes|pseudo-genome|9653|Chr2:8278881..8286445|Col-0_gi:AT2G19110.1|V0.2', description='MPI-GMI|Ath-1001-Genomes|pseudo-genome|9653|Chr2:8278881..8286445|Col-0_gi:AT2G19110.1|V0.2', dbxrefs=[])

In [ ]:
aa = record.seq.translate()

/usr/local/lib/python3.12/dist-packages/Bio/Seq.py:2877: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [ ]:
start, _ = get_start_end_from_seq(record)
start

8278881

In [ ]:
in_gene_position = first_result["position"] - start

In [ ]:
first_result["variant"] == record.seq[in_gene_position]

True

In [ ]:
codon_start = (in_gene_position // 3) * 3
codon_start

642

In [ ]:
in_condon_position = in_gene_position % 3
in_condon_position

0

In [ ]:
codon = MutableSeq(record.seq[codon_start:codon_start+3])

In [ ]:
codon.translate()

MutableSeq('V')

In [ ]:
codon[in_condon_position] = first_result["reference"]
codon.translate()

MutableSeq('L')

In [ ]:
aa

Seq('LRS*XFSQPLSDRTKPVFSHRLLPFAXSSPLFSGXFVSLLNSHRFHDK**SIXR...CNS')

In [ ]:
record.id

'MPI-GMI|Ath-1001-Genomes|pseudo-genome|9653|Chr2:8278881..8286445|Col-0_gi:AT2G19110.1|V0.2'

In [ ]:
record.seq[in_gene_position]

'G'

In [ ]:
snp_centered_dna_window = record.seq[in_gene_position-256:in_gene_position+257]

In [ ]:
len(dna_snippet)

513

In [ ]:
dna_snippet[len(dna_snippet) // 2]

'G'

In [ ]:
DefaultDict(lambda key: key + 5)

In [ ]:
foo = "AAATAAA"

foo[len(foo) // 2]

'T'

In [ ]:
foo[3-10:3+2]

'AAATA'